In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [2]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,8).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [3]:
# run optimization with constraints
QMLE_scipy_estimate(alpha, c0, x1, y1, W_ls,initial_guess = params)

/Users/helangchun/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[0.2793975067278372,
 -0.15305529362385853,
 0.48265964326800814,
 2.34921430113097,
 array([0.98424237, 0.96434712])]

In [4]:
QMLE_scipy_estimate(alpha, c0, x1, y1, W_ls,initial_guess = params, constrain= False)

[0.2793979753954539,
 -0.1530541536374055,
 0.48266385546265134,
 2.3492212112937207,
 array([0.98424602, 0.9643478 ])]

### 2. Test k=1

In [5]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 49
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [6]:
QMLE_scipy_estimate(alpha, c0, x, y, W_ls,initial_guess = initial_guess, constrain= False)

[0.9215231765447613,
 0.4922398758847577,
 -0.00914798749363685,
 0.24341736537097053,
 array([0.93797834])]